# fcecodec-example for Google Colab

<a href="https://colab.research.google.com/github/bfut/notebooks/blob/main/fcecodec/fcecodec-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a basic example.

https://github.com/bfut/fcecodec</br>
https://github.com/bfut/notebooks ```/fcecodec/fcecodec-example.ipynb```

In [ ]:
!ls -lg
!rm -rf fcecodec

In [ ]:
!python -m pip install --upgrade pip wheel setuptools pybind11
!git clone https://github.com/bfut/fcecodec
!python -m pip install --verbose fcecodec/python

Install ```wurlitzer``` which captures console output of library functions. This is only necessary for Google Colab:

In [ ]:
# https://github.com/minrk/wurlitzer
# Capture C-level stdout/stderr
!pip install wurlitzer
from wurlitzer import pipes

# with pipes() as (out, err):
#     call_some_c_function()
# print(out.read())

In [ ]:
import fcecodec
help(fcecodec)  # smoketest

In [ ]:
filepath_fce_input = "fcecodec/tests/fce/Snowman_car.fce"
filepath_fce_output = "fcecodec/tests/fce/car_output.fce"
filepath_fce_input2 = "fcecodec/tests/fce/Snowman_car.fce"

In [ ]:
with pipes() as (out, err):
    with open(filepath_fce_input, "rb") as f:
        fce_buf = f.read()

    # Print FCE stats
    fcecodec.PrintFceInfo(fce_buf)
print(out.read())

In [ ]:
# Create Mesh object
mesh = fcecodec.Mesh()

# Load FCE data to Mesh object
mesh.IoDecode(fce_buf)

with pipes() as (out, err):
    # Print Mesh object stats
    print(mesh.MNumParts)
    print(mesh.MNumTriags)
    print(mesh.MNumVerts)
    mesh.PrintInfo()

    # Validate Mesh object
    assert(mesh.MValid() == 1)
print(out.read())

In [ ]:
# Merge parts 0, 3 to new part
new_pid = mesh.OpMergeParts(0, 3)
assert(new_pid != -1)

In [ ]:
# Copy part 1
new_pid = mesh.OpCopyPart(1)
assert(new_pid != -1)

In [ ]:
with pipes() as (out, err):
    # Print Mesh object stats
    print(mesh.MNumParts)
    print(mesh.MNumTriags)
    print(mesh.MNumVerts)
    mesh.PrintInfo()

    # Validate Mesh object
    assert(mesh.MValid() == 1)
print(out.read())

In [ ]:
# Insert/copy part 1 from mesh_src to mesh
with open(filepath_fce_input2, "rb") as f:
    fce_buf2 = f.read()
mesh_src = fcecodec.Mesh()
mesh_src.IoDecode(fce_buf2)
new_pid = mesh.OpInsertPart(mesh_src, 1)
assert(new_pid != -1)

In [ ]:
# Encode to FCE4M
out_buf = mesh.IoEncode_Fce4M()
with open(filepath_fce_output, "wb") as f:
    f.write(out_buf)

!cd fcecodec/tests/fce; ls -lg

In [ ]:
with pipes() as (out, err):
    with open(filepath_fce_output, "rb") as f:
        fce_buf2 = f.read()

    # Print FCE stats
    fcecodec.PrintFceInfo(fce_buf2)
print(out.read())